<a href="https://colab.research.google.com/github/AntonioCuba123/skills-introduction-to-github/blob/main/Modelo_de_predicci%C3%B3n_y_Script_de_validaci%C3%B3n_Canasta_de_productos_supermercado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Market Basket Analysis Predictivo para Supermercado
# Autor: Claude
# Fecha: 16 de abril de 2025

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.multioutput import MultiOutputClassifier
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(42)

# 1. GENERACIÓN DE DATOS SIMULADOS
# ------------------------------

# Definimos categorías y productos
categorias = {
    'Lácteos': ['Leche', 'Yogurt', 'Queso', 'Mantequilla', 'Crema'],
    'Panadería': ['Pan', 'Galletas', 'Pasteles', 'Tortillas', 'Pan integral'],
    'Carnes': ['Pollo', 'Res', 'Cerdo', 'Jamón', 'Salchicha'],
    'Frutas y Verduras': ['Manzana', 'Plátano', 'Tomate', 'Cebolla', 'Zanahoria'],
    'Bebidas': ['Agua', 'Refresco', 'Jugo', 'Cerveza', 'Café'],
    'Limpieza': ['Jabón', 'Detergente', 'Cloro', 'Papel higiénico', 'Servilletas'],
    'Despensa': ['Arroz', 'Frijol', 'Azúcar', 'Aceite', 'Pasta']
}

# Lista plana de todos los productos
todos_productos = [producto for lista in categorias.values() for producto in lista]

# Parámetros de simulación
n_clientes = 500
n_transacciones = 10000
fecha_inicio = datetime(2024, 1, 1)
fecha_fin = datetime(2024, 10, 31)

# Simular datos de clientes
clientes = []

segmentos = ['Hogar joven', 'Adulto soltero', 'Familia con niños', 'Adulto mayor', 'Pareja sin hijos']
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

for i in range(1, n_clientes + 1):
    edad = random.randint(18, 85)
    # Asignar segmento según la edad (con algo de aleatoriedad)
    if 18 <= edad <= 30:
        segmento = random.choices(segmentos, weights=[0.5, 0.3, 0.05, 0.05, 0.1])[0]
    elif 31 <= edad <= 45:
        segmento = random.choices(segmentos, weights=[0.3, 0.2, 0.3, 0.05, 0.15])[0]
    elif 46 <= edad <= 65:
        segmento = random.choices(segmentos, weights=[0.05, 0.1, 0.3, 0.2, 0.35])[0]
    else:
        segmento = random.choices(segmentos, weights=[0.05, 0.1, 0.05, 0.7, 0.1])[0]

    frecuencia_visitas = round(random.uniform(1, 15), 1)  # Entre 1 y 15 visitas al mes
    gasto_promedio = round(random.uniform(200, 3000), 2)  # Entre $200 y $3000 MXN
    usa_cupones = random.choices([0, 1], weights=[0.6, 0.4])[0]

    clientes.append({
        'id_cliente': f'C{i:03d}',
        'edad': edad,
        'segmento': segmento,
        'frecuencia_visitas_mes': frecuencia_visitas,
        'gasto_promedio_mensual': gasto_promedio,
        'usa_cupones': usa_cupones
    })

df_clientes = pd.DataFrame(clientes)

# Generar transacciones
transacciones = []

# Función para generar canasta según segmento y hora
def generar_canasta(segmento, hora, usa_cupones):
    n_productos = random.randint(1, 15)  # Entre 1 y 15 productos por compra

    # Probabilidades por categoría según segmento
    prob_categorias = {
        'Hogar joven': {'Lácteos': 0.6, 'Panadería': 0.7, 'Carnes': 0.4, 'Frutas y Verduras': 0.5, 'Bebidas': 0.8, 'Limpieza': 0.4, 'Despensa': 0.5},
        'Adulto soltero': {'Lácteos': 0.5, 'Panadería': 0.6, 'Carnes': 0.3, 'Frutas y Verduras': 0.4, 'Bebidas': 0.9, 'Limpieza': 0.3, 'Despensa': 0.4},
        'Familia con niños': {'Lácteos': 0.9, 'Panadería': 0.8, 'Carnes': 0.7, 'Frutas y Verduras': 0.7, 'Bebidas': 0.6, 'Limpieza': 0.8, 'Despensa': 0.9},
        'Adulto mayor': {'Lácteos': 0.7, 'Panadería': 0.6, 'Carnes': 0.5, 'Frutas y Verduras': 0.8, 'Bebidas': 0.3, 'Limpieza': 0.6, 'Despensa': 0.7},
        'Pareja sin hijos': {'Lácteos': 0.6, 'Panadería': 0.7, 'Carnes': 0.6, 'Frutas y Verduras': 0.7, 'Bebidas': 0.8, 'Limpieza': 0.5, 'Despensa': 0.6}
    }

    # Modificar probabilidades según hora del día
    if hora == 'Mañana':
        prob_categorias[segmento]['Panadería'] += 0.2
        prob_categorias[segmento]['Lácteos'] += 0.1
    elif hora == 'Tarde':
        prob_categorias[segmento]['Frutas y Verduras'] += 0.1
        prob_categorias[segmento]['Carnes'] += 0.1
    else:  # Noche
        prob_categorias[segmento]['Bebidas'] += 0.1
        prob_categorias[segmento]['Limpieza'] -= 0.1

    # Bonus de probabilidad si usa cupones
    if usa_cupones:
        for cat in prob_categorias[segmento]:
            prob_categorias[segmento][cat] = min(prob_categorias[segmento][cat] + 0.1, 0.95)

    # Seleccionar categorías según probabilidad
    categorias_seleccionadas = []
    for cat, prob in prob_categorias[segmento].items():
        if random.random() < prob:
            categorias_seleccionadas.append(cat)

    # Si no hay categorías, seleccionar al menos una
    if not categorias_seleccionadas:
        categorias_seleccionadas = [random.choice(list(categorias.keys()))]

    # Seleccionar productos de las categorías elegidas
    canasta = []
    while len(canasta) < n_productos:
        cat = random.choice(categorias_seleccionadas)
        prod = random.choice(categorias[cat])
        if prod not in canasta:  # Evitar duplicados
            canasta.append(prod)

    return canasta

# Generar transacciones
for i in range(n_transacciones):
    # Seleccionar un cliente aleatorio
    cliente = random.choice(clientes)
    id_cliente = cliente['id_cliente']
    segmento = cliente['segmento']
    usa_cupones = cliente['usa_cupones']

    # Generar fecha y hora
    dias_aleatorios = random.randint(0, (fecha_fin - fecha_inicio).days)
    fecha = fecha_inicio + timedelta(days=dias_aleatorios)
    hora_categoria = random.choices(['Mañana', 'Tarde', 'Noche'], weights=[0.3, 0.4, 0.3])[0]

    # Días desde última compra (aleatorio para simular)
    dias_ultima_compra = random.randint(1, 30)

    # Generar canasta
    canasta = generar_canasta(segmento, hora_categoria, usa_cupones)

    # Calcular monto de compra (relación con cantidad de productos y gasto promedio)
    gasto_base = cliente['gasto_promedio_mensual'] / cliente['frecuencia_visitas_mes']
    factor_aleatorio = random.uniform(0.5, 1.5)
    factor_productos = min(1 + (len(canasta) / 20), 1.8)  # Factor según cantidad de productos
    monto_compra = round(gasto_base * factor_productos * factor_aleatorio, 2)

    # Día de la semana
    dia_semana = dias_semana[fecha.weekday()]

    transacciones.append({
        'id_transaccion': f'T{i+1:05d}',
        'id_cliente': id_cliente,
        'fecha_compra': fecha.strftime('%Y-%m-%d'),
        'hora_compra': hora_categoria,
        'dia_semana': dia_semana,
        'productos_comprados': ','.join(canasta),
        'monto_compra': monto_compra,
        'dias_desde_ultima_compra': dias_ultima_compra
    })

df_transacciones = pd.DataFrame(transacciones)

# 2. PREPARACIÓN DE DATOS PARA MODELADO
# -----------------------------------

# Ordenar transacciones por cliente y fecha para simular secuencia temporal
df_transacciones['fecha_compra'] = pd.to_datetime(df_transacciones['fecha_compra'])
df_transacciones = df_transacciones.sort_values(by=['id_cliente', 'fecha_compra'])

# Crear conjuntos de entrenamiento y predicción
# Usaremos N-1 compras para predecir la N-ésima compra
cliente_grupos = df_transacciones.groupby('id_cliente')

X_data = []  # Características para el modelo
y_data = []  # Objetivo: próxima canasta

# Procesar las transacciones para crear datos de entrenamiento
for id_cliente, grupo in cliente_grupos:
    if len(grupo) < 2:  # Necesitamos al menos 2 compras para entrenar y validar
        continue

    # Obtener datos del cliente
    cliente_info = df_clientes[df_clientes['id_cliente'] == id_cliente].iloc[0]

    # Para cada compra (excepto la última), predecir la siguiente
    for i in range(len(grupo) - 1):
        compra_actual = grupo.iloc[i]
        compra_siguiente = grupo.iloc[i + 1]

        # Crear vector de características
        features = {
            'id_cliente': id_cliente,
            'edad': cliente_info['edad'],
            'segmento': cliente_info['segmento'],
            'frecuencia_visitas_mes': cliente_info['frecuencia_visitas_mes'],
            'gasto_promedio_mensual': cliente_info['gasto_promedio_mensual'],
            'usa_cupones': cliente_info['usa_cupones'],
            'hora_compra': compra_actual['hora_compra'],
            'dia_semana': compra_actual['dia_semana'],
            'monto_compra': compra_actual['monto_compra'],
            'dias_desde_ultima_compra': compra_actual['dias_desde_ultima_compra'],
            'productos_comprados': compra_actual['productos_comprados']
        }

        # Target: productos de la próxima compra
        target = compra_siguiente['productos_comprados'].split(',')

        X_data.append(features)
        y_data.append(target)

# Convertir a DataFrame
df_X = pd.DataFrame(X_data)
df_y = pd.DataFrame(y_data)

# 3. INGENIERÍA DE CARACTERÍSTICAS
# ------------------------------

# One-hot encoding para productos comprados anteriormente
def crear_one_hot_productos(productos_str, todos_productos):
    productos_list = productos_str.split(',')
    return {producto: 1 if producto in productos_list else 0 for producto in todos_productos}

# Aplicar one-hot encoding a productos comprados
productos_one_hot = df_X['productos_comprados'].apply(lambda x: crear_one_hot_productos(x, todos_productos))
df_productos_one_hot = pd.DataFrame(productos_one_hot.tolist())

# One-hot encoding para variables categóricas
df_X_encoded = pd.get_dummies(df_X.drop('productos_comprados', axis=1),
                             columns=['id_cliente', 'segmento', 'hora_compra', 'dia_semana'])

# Unir con one-hot de productos
df_X_final = pd.concat([df_X_encoded, df_productos_one_hot], axis=1)

# Codificar target
def crear_target_vector(productos_list, todos_productos):
    return [1 if producto in productos_list else 0 for producto in todos_productos]

y_encoded = np.array([crear_target_vector(target, todos_productos) for target in y_data])

# 4. ENTRENAMIENTO DEL MODELO
# -------------------------

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_X_final, y_encoded, test_size=0.2, random_state=42)

# Escalar características numéricas
scaler = StandardScaler()
numeric_cols = ['edad', 'frecuencia_visitas_mes', 'gasto_promedio_mensual', 'monto_compra', 'dias_desde_ultima_compra']
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Entrenar modelo Multi-output RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_rf = MultiOutputClassifier(rf_model)

print("Entrenando modelo...")
multi_target_rf.fit(X_train, y_train)
print("Modelo entrenado correctamente.")

# 5. EVALUACIÓN DEL MODELO
# ----------------------

# Predecir en conjunto de prueba
y_pred = multi_target_rf.predict(X_test)

# Calcular métricas por producto
precision_por_producto = []
recall_por_producto = []
f1_por_producto = []

for i, producto in enumerate(todos_productos):
    precision = precision_score(y_test[:, i], y_pred[:, i], zero_division=0)
    recall = recall_score(y_test[:, i], y_pred[:, i], zero_division=0)
    f1 = f1_score(y_test[:, i], y_pred[:, i], zero_division=0)

    precision_por_producto.append((producto, precision))
    recall_por_producto.append((producto, recall))
    f1_por_producto.append((producto, f1))

# Ordenar por F1-score
f1_por_producto.sort(key=lambda x: x[1], reverse=True)

print("\nTop 10 productos con mejor predicción (F1-score):")
for producto, f1 in f1_por_producto[:10]:
    print(f"{producto}: {f1:.4f}")

# Métricas globales
precision_global = precision_score(y_test.flatten(), y_pred.flatten(), zero_division=0)
recall_global = recall_score(y_test.flatten(), y_pred.flatten(), zero_division=0)
f1_global = f1_score(y_test.flatten(), y_pred.flatten(), zero_division=0)

print(f"\nMétricas globales:")
print(f"Precision: {precision_global:.4f}")
print(f"Recall: {recall_global:.4f}")
print(f"F1-score: {f1_global:.4f}")

# 6. ANÁLISIS DE REGLAS DE ASOCIACIÓN
# ---------------------------------

# Preparar datos para reglas de asociación
def crear_matriz_transacciones(df):
    # Crear matriz one-hot de productos por transacción
    lista_productos = [row.split(',') for row in df['productos_comprados']]
    transacciones = pd.DataFrame({
        'id_transaccion': df['id_transaccion'].repeat([len(p) for p in lista_productos]),
        'producto': [item for sublist in lista_productos for item in sublist]
    })

    return pd.crosstab(transacciones['id_transaccion'], transacciones['producto'])

# Convertir transacciones a formato one-hot
matriz_transacciones = crear_matriz_transacciones(df_transacciones)

# Ejecutar algoritmo Apriori
frequent_itemsets = apriori(matriz_transacciones, min_support=0.01, use_colnames=True)

# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

if len(rules) > 0:
    # Ordenar por lift
    rules_sorted = rules.sort_values('lift', ascending=False)

    print("\nTop 10 reglas de asociación (lift):")
    for i, row in rules_sorted.head(10).iterrows():
        antecedentes = list(row['antecedents'])
        consecuentes = list(row['consequents'])
        print(f"Regla: {antecedentes} -> {consecuentes}")
        print(f"   Support: {row['support']:.4f}, Confidence: {row['confidence']:.4f}, Lift: {row['lift']:.4f}")
else:
    print("\nNo se encontraron reglas de asociación con los parámetros actuales.")

# 7. EJEMPLO DE PREDICCIÓN
# ---------------------

# Seleccionar un cliente aleatorio para predicción
cliente_demo = random.choice(df_clientes['id_cliente'].tolist())
ultimas_compras = df_transacciones[df_transacciones['id_cliente'] == cliente_demo].sort_values('fecha_compra').tail(3)

print(f"\nCliente seleccionado para demostración: {cliente_demo}")
print("Últimas 3 compras:")
for i, row in ultimas_compras.iterrows():
    print(f"Fecha: {row['fecha_compra'].strftime('%Y-%m-%d')}, Productos: {row['productos_comprados']}")

# Preparar datos para predicción
cliente_info = df_clientes[df_clientes['id_cliente'] == cliente_demo].iloc[0]
ultima_compra = ultimas_compras.iloc[-1]

# Crear vector de características
features_demo = {
    'id_cliente': cliente_demo,
    'edad': cliente_info['edad'],
    'segmento': cliente_info['segmento'],
    'frecuencia_visitas_mes': cliente_info['frecuencia_visitas_mes'],
    'gasto_promedio_mensual': cliente_info['gasto_promedio_mensual'],
    'usa_cupones': cliente_info['usa_cupones'],
    'hora_compra': ultima_compra['hora_compra'],
    'dia_semana': ultima_compra['dia_semana'],
    'monto_compra': ultima_compra['monto_compra'],
    'dias_desde_ultima_compra': ultima_compra['dias_desde_ultima_compra'],
    'productos_comprados': ultima_compra['productos_comprados']
}

# Convertir a DataFrame
df_demo = pd.DataFrame([features_demo])

# Aplicar one-hot encoding
productos_one_hot_demo = df_demo['productos_comprados'].apply(lambda x: crear_one_hot_productos(x, todos_productos))
df_productos_one_hot_demo = pd.DataFrame(productos_one_hot_demo.tolist())

# One-hot encoding para variables categóricas
df_demo_encoded = pd.get_dummies(df_demo.drop('productos_comprados', axis=1),
                                columns=['id_cliente', 'segmento', 'hora_compra', 'dia_semana'])

# Alinear columnas con el conjunto de entrenamiento
for col in df_X_final.columns:
    if col not in df_demo_encoded.columns and col not in df_productos_one_hot_demo.columns:
        df_demo_encoded[col] = 0

# Unir one-hot de productos
missing_producto_cols = set(df_productos_one_hot.columns) - set(df_productos_one_hot_demo.columns)
for col in missing_producto_cols:
    df_productos_one_hot_demo[col] = 0

df_demo_final = pd.concat([df_demo_encoded, df_productos_one_hot_demo], axis=1)
df_demo_final = df_demo_final[df_X_final.columns]  # Asegurar mismo orden de columnas

# Escalar características numéricas
df_demo_final[numeric_cols] = scaler.transform(df_demo_final[numeric_cols])

# Predecir próxima canasta
prediccion = multi_target_rf.predict(df_demo_final)

# Mostrar productos predichos
productos_predichos = [todos_productos[i] for i, pred in enumerate(prediccion[0]) if pred == 1]

print("\nProductos predichos para próxima compra:")
print(productos_predichos)

# 8. VISUALIZACIONES
# ---------------

# Gráfica 1: Top 10 productos más comprados
productos_conteo = []
for productos_str in df_transacciones['productos_comprados']:
    productos_conteo.extend(productos_str.split(','))

df_productos_count = pd.Series(productos_conteo).value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=df_productos_count.values, y=df_productos_count.index)
plt.title('Top 10 Productos Más Comprados')
plt.xlabel('Cantidad de Compras')
plt.tight_layout()
plt.savefig('top_10_productos.png')

# Gráfica 2: Distribución de compras por hora del día
plt.figure(figsize=(8, 5))
sns.countplot(data=df_transacciones, x='hora_compra', order=['Mañana', 'Tarde', 'Noche'])
plt.title('Distribución de Compras por Hora del Día')
plt.xlabel('Hora del Día')
plt.ylabel('Cantidad de Transacciones')
plt.tight_layout()
plt.savefig('compras_por_hora.png')

# Gráfica 3: Importancia de características (top 20)
importancias = []
for i, estimador in enumerate(multi_target_rf.estimators_):
    importancias.append(estimador.feature_importances_)

importancia_promedio = np.mean(importancias, axis=0)
indices = np.argsort(importancia_promedio)[-20:]  # Top 20 características

plt.figure(figsize=(10, 8))
sns.barplot(x=importancia_promedio[indices], y=np.array(df_X_final.columns)[indices])
plt.title('Top 20 Características Más Importantes')
plt.xlabel('Importancia Promedio')
plt.tight_layout()
plt.savefig('importancia_caracteristicas.png')

print("\nAnálisis completado. Se han generado visualizaciones.")

# 9. AGRUPAMIENTO DE CLIENTES (CLUSTERING)
# -------------------------------------

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Seleccionar características para clustering
df_clustering = df_clientes.copy()

# Codificar segmentos
le_segmento = LabelEncoder()
df_clustering['segmento_cod'] = le_segmento.fit_transform(df_clustering['segmento'])

# Seleccionar características numéricas
features_cluster = ['edad', 'frecuencia_visitas_mes', 'gasto_promedio_mensual', 'usa_cupones', 'segmento_cod']
X_cluster = df_clustering[features_cluster].values

# Escalar datos
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# Aplicar PCA para visualización
pca = PCA(n_components=2)
X_cluster_pca = pca.fit_transform(X_cluster_scaled)

# Determinar número óptimo de clusters (método del codo)
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Método del Codo para Número Óptimo de Clusters')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.grid(True)
plt.savefig('metodo_codo.png')

# Aplicar K-means con número óptimo de clusters (4 para este ejemplo)
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_cluster_scaled)

# Agregar clusters al DataFrame
df_clustering['cluster'] = clusters

# Visualizar clusters
plt.figure(figsize=(10, 8))
for cluster in range(n_clusters):
    plt.scatter(X_cluster_pca[clusters == cluster, 0],
                X_cluster_pca[clusters == cluster, 1],
                label=f'Cluster {cluster}')

plt.title('Segmentación de Clientes (PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.grid(True)
plt.savefig('clusters_pca.png')

# Características de cada cluster
cluster_stats = df_clustering.groupby('cluster').agg({
    'edad': 'mean',
    'frecuencia_visitas_mes': 'mean',
    'gasto_promedio_mensual': 'mean',
    'usa_cupones': 'mean',
    'id_cliente': 'count'
}).reset_index()

cluster_stats = cluster_stats.rename(columns={'id_cliente': 'num_clientes'})

print("\nEstadísticas por Cluster:")
print(cluster_stats)

# Análisis de segmentos por cluster
segmento_por_cluster = pd.crosstab(df_clustering['cluster'], df_clustering['segmento'], normalize='index')
print("\nDistribución de segmentos por cluster (%):")
print(segmento_por_cluster * 100)

# 10. RECOMENDACIONES PERSONALIZADAS POR CLUSTER
# -------------------------------------------

# Obtener productos típicos por cluster
productos_por_cluster = []

for cluster in range(n_clusters):
    # Obtener IDs de clientes en este cluster
    clientes_cluster = df_clustering[df_clustering['cluster'] == cluster]['id_cliente'].tolist()

    # Obtener todas las transacciones de estos clientes
    transacciones_cluster = df_transacciones[df_transacciones['id_cliente'].isin(clientes_cluster)]

    # Contar productos
    productos_conteo = []
    for productos_str in transacciones_cluster['productos_comprados']:
        productos_conteo.extend(productos_str.split(','))

    # Top 5 productos más comprados en este cluster
    top_productos = pd.Series(productos_conteo).value_counts().head(5)

    productos_por_cluster.append(top_productos)

print("\nProductos populares por cluster:")
for i, productos in enumerate(productos_por_cluster):
    print(f"\nCluster {i}:")
    for producto, count in productos.items():
        print(f"  {producto}: {count}")

print("\nAnálisis de clustering completado.")

# 11. GUARDAR RESULTADOS
# -------------------

# Guardar datasets
df_clientes.to_csv('clientes_simulados.csv', index=False)
df_transacciones.to_csv('transacciones_simuladas.csv', index=False)

print("\nDataset guardado en archivos CSV.")
print("Modelo de predicción de canasta de compras completado.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###SCRIPT DE EVALUACIÓN DEL MODELO

In [ ]:
# Script de Evaluación del Modelo Predictivo de Canasta de Compras
# Autor: Claude
# Fecha: 16 de abril de 2025

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score, KFold
import joblib
import random
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(42)

# 1. FUNCIONES DE CARGA DE DATOS Y MODELO
# -------------------------------------

def cargar_datos(ruta_clientes='clientes_simulados.csv', ruta_transacciones='transacciones_simuladas.csv'):
    """Carga los datasets de clientes y transacciones"""
    df_clientes = pd.read_csv(ruta_clientes)
    df_transacciones = pd.read_csv(ruta_transacciones)
    df_transacciones['fecha_compra'] = pd.to_datetime(df_transacciones['fecha_compra'])

    return df_clientes, df_transacciones

def obtener_categorias_productos():
    """Define las categorías y productos usados en el dataset"""
    categorias = {
        'Lácteos': ['Leche', 'Yogurt', 'Queso', 'Mantequilla', 'Crema'],
        'Panadería': ['Pan', 'Galletas', 'Pasteles', 'Tortillas', 'Pan integral'],
        'Carnes': ['Pollo', 'Res', 'Cerdo', 'Jamón', 'Salchicha'],
        'Frutas y Verduras': ['Manzana', 'Plátano', 'Tomate', 'Cebolla', 'Zanahoria'],
        'Bebidas': ['Agua', 'Refresco', 'Jugo', 'Cerveza', 'Café'],
        'Limpieza': ['Jabón', 'Detergente', 'Cloro', 'Papel higiénico', 'Servilletas'],
        'Despensa': ['Arroz', 'Frijol', 'Azúcar', 'Aceite', 'Pasta']
    }
    todos_productos = [producto for lista in categorias.values() for producto in lista]

    return categorias, todos_productos

def cargar_modelo(ruta_modelo='modelo_basket_prediction.pkl'):
    """Carga el modelo entrenado desde el archivo"""
    try:
        modelo = joblib.load(ruta_modelo)
        print("Modelo cargado correctamente")
        return modelo
    except:
        print("No se encontró el modelo guardado. Por favor, entrene el modelo primero.")
        return None

# También puedes entrenar el modelo desde cero si no tienes uno guardado
def entrenar_nuevo_modelo(X_train, y_train):
    """Entrena un nuevo modelo de predicción de canasta"""
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.multioutput import MultiOutputClassifier

    print("Entrenando nuevo modelo...")
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    multi_target_rf = MultiOutputClassifier(rf_model)
    multi_target_rf.fit(X_train, y_train)
    print("Modelo entrenado correctamente")

    # Guardar el modelo
    joblib.dump(multi_target_rf, 'modelo_basket_prediction.pkl')

    return multi_target_rf

# 2. FUNCIONES DE PREPROCESAMIENTO
# -----------------------------

def crear_one_hot_productos(productos_str, todos_productos):
    """Convierte cadena de productos a vector one-hot"""
    productos_list = productos_str.split(',')
    return {producto: 1 if producto in productos_list else 0 for producto in todos_productos}

def crear_target_vector(productos_list, todos_productos):
    """Convierte lista de productos a vector binario"""
    return [1 if producto in productos_list else 0 for producto in todos_productos]

def preparar_datos_modelado(df_clientes, df_transacciones, todos_productos):
    """Prepara los datos para modelado como se hizo en el script original"""
    # Ordenar transacciones
    df_transacciones = df_transacciones.sort_values(by=['id_cliente', 'fecha_compra'])

    # Agrupar por cliente
    cliente_grupos = df_transacciones.groupby('id_cliente')

    X_data = []  # Características
    y_data = []  # Objetivo

    # Procesar transacciones para crear datos de entrenamiento
    for id_cliente, grupo in cliente_grupos:
        if len(grupo) < 2:  # Necesitamos al menos 2 compras para entrenar y validar
            continue

        # Obtener datos del cliente
        cliente_info = df_clientes[df_clientes['id_cliente'] == id_cliente].iloc[0]

        # Para cada compra (excepto la última), predecir la siguiente
        for i in range(len(grupo) - 1):
            compra_actual = grupo.iloc[i]
            compra_siguiente = grupo.iloc[i + 1]

            # Crear vector de características
            features = {
                'id_cliente': id_cliente,
                'edad': cliente_info['edad'],
                'segmento': cliente_info['segmento'],
                'frecuencia_visitas_mes': cliente_info['frecuencia_visitas_mes'],
                'gasto_promedio_mensual': cliente_info['gasto_promedio_mensual'],
                'usa_cupones': cliente_info['usa_cupones'],
                'hora_compra': compra_actual['hora_compra'],
                'dia_semana': compra_actual['dia_semana'],
                'monto_compra': compra_actual['monto_compra'],
                'dias_desde_ultima_compra': compra_actual['dias_desde_ultima_compra'],
                'productos_comprados': compra_actual['productos_comprados']
            }

            # Target: productos de la próxima compra
            target = compra_siguiente['productos_comprados'].split(',')

            X_data.append(features)
            y_data.append(target)

    # Convertir a DataFrame
    df_X = pd.DataFrame(X_data)

    # One-hot encoding para productos comprados
    productos_one_hot = df_X['productos_comprados'].apply(lambda x: crear_one_hot_productos(x, todos_productos))
    df_productos_one_hot = pd.DataFrame(productos_one_hot.tolist())

    # One-hot encoding para variables categóricas
    df_X_encoded = pd.get_dummies(df_X.drop('productos_comprados', axis=1),
                                 columns=['id_cliente', 'segmento', 'hora_compra', 'dia_semana'])

    # Unir con one-hot de productos
    df_X_final = pd.concat([df_X_encoded, df_productos_one_hot], axis=1)

    # Codificar target
    y_encoded = np.array([crear_target_vector(target, todos_productos) for target in y_data])

    return df_X_final, y_encoded

def escalar_datos(X_train, X_test, numeric_cols):
    """Escala las características numéricas"""
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

    return X_train, X_test, scaler

# 3. FUNCIONES DE EVALUACIÓN DEL MODELO
# ----------------------------------

def evaluar_modelo_global(modelo, X_test, y_test):
    """Evalúa el rendimiento global del modelo"""
    y_pred = modelo.predict(X_test)

    # Calcular métricas globales
    precision_global = precision_score(y_test.flatten(), y_pred.flatten(), zero_division=0)
    recall_global = recall_score(y_test.flatten(), y_pred.flatten(), zero_division=0)
    f1_global = f1_score(y_test.flatten(), y_pred.flatten(), zero_division=0)
    accuracy_global = accuracy_score(y_test.flatten(), y_pred.flatten())

    print("\n==== EVALUACIÓN GLOBAL DEL MODELO ====")
    print(f"Accuracy: {accuracy_global:.4f}")
    print(f"Precision: {precision_global:.4f}")
    print(f"Recall: {recall_global:.4f}")
    print(f"F1-score: {f1_global:.4f}")

    return {
        'accuracy': accuracy_global,
        'precision': precision_global,
        'recall': recall_global,
        'f1': f1_global
    }

def evaluar_modelo_por_producto(modelo, X_test, y_test, todos_productos):
    """Evalúa el rendimiento del modelo por producto"""
    y_pred = modelo.predict(X_test)

    resultados_por_producto = []

    for i, producto in enumerate(todos_productos):
        precision = precision_score(y_test[:, i], y_pred[:, i], zero_division=0)
        recall = recall_score(y_test[:, i], y_pred[:, i], zero_division=0)
        f1 = f1_score(y_test[:, i], y_pred[:, i], zero_division=0)

        # Calcular matriz de confusión
        tn, fp, fn, tp = confusion_matrix(y_test[:, i], y_pred[:, i]).ravel()

        resultados_por_producto.append({
            'producto': producto,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'true_positives': tp,
            'false_positives': fp,
            'true_negatives': tn,
            'false_negatives': fn
        })

    # Convertir a DataFrame para mejor visualización
    df_resultados = pd.DataFrame(resultados_por_producto)

    # Ordenar por F1-score
    df_resultados_sorted = df_resultados.sort_values('f1', ascending=False)

    print("\n==== TOP 10 PRODUCTOS CON MEJOR PREDICCIÓN (F1-SCORE) ====")
    print(df_resultados_sorted[['producto', 'precision', 'recall', 'f1']].head(10))

    print("\n==== 10 PRODUCTOS CON PEOR PREDICCIÓN (F1-SCORE) ====")
    print(df_resultados_sorted[['producto', 'precision', 'recall', 'f1']].tail(10))

    return df_resultados_sorted

def evaluar_modelo_por_categoria(modelo, X_test, y_test, todos_productos, categorias):
    """Evalúa el rendimiento del modelo por categoría de producto"""
    y_pred = modelo.predict(X_test)

    resultados_por_categoria = []

    # Para cada categoría, evaluar el rendimiento promedio de sus productos
    for categoria, productos in categorias.items():
        # Obtener índices de los productos en esta categoría
        indices = [todos_productos.index(producto) for producto in productos]

        # Calcular métricas para esta categoría
        precision_cat = precision_score(y_test[:, indices].flatten(), y_pred[:, indices].flatten(), zero_division=0)
        recall_cat = recall_score(y_test[:, indices].flatten(), y_pred[:, indices].flatten(), zero_division=0)
        f1_cat = f1_score(y_test[:, indices].flatten(), y_pred[:, indices].flatten(), zero_division=0)

        resultados_por_categoria.append({
            'categoria': categoria,
            'precision': precision_cat,
            'recall': recall_cat,
            'f1': f1_cat
        })

    df_resultados_cat = pd.DataFrame(resultados_por_categoria)
    df_resultados_cat_sorted = df_resultados_cat.sort_values('f1', ascending=False)

    print("\n==== RENDIMIENTO POR CATEGORÍA ====")
    print(df_resultados_cat_sorted)

    return df_resultados_cat_sorted

def validacion_cruzada(X, y, n_splits=5):
    """Realiza validación cruzada del modelo"""
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.multioutput import MultiOutputClassifier

    print("\n==== VALIDACIÓN CRUZADA ====")

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    precision_scores = []
    recall_scores = []
    f1_scores = []

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        print(f"\nEntrenando fold {fold+1}/{n_splits}...")

        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Escalar características numéricas
        numeric_cols = ['edad', 'frecuencia_visitas_mes', 'gasto_promedio_mensual', 'monto_compra', 'dias_desde_ultima_compra']
        scaler = StandardScaler()
        X_train_fold[numeric_cols] = scaler.fit_transform(X_train_fold[numeric_cols])
        X_test_fold[numeric_cols] = scaler.transform(X_test_fold[numeric_cols])

        # Entrenar modelo
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        multi_target_rf = MultiOutputClassifier(rf_model)
        multi_target_rf.fit(X_train_fold, y_train_fold)

        # Predecir
        y_pred_fold = multi_target_rf.predict(X_test_fold)

        # Calcular métricas
        precision = precision_score(y_test_fold.flatten(), y_pred_fold.flatten(), zero_division=0)
        recall = recall_score(y_test_fold.flatten(), y_pred_fold.flatten(), zero_division=0)
        f1 = f1_score(y_test_fold.flatten(), y_pred_fold.flatten(), zero_division=0)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold+1} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    print("\nResultados promedio de validación cruzada:")
    print(f"Precision: {np.mean(precision_scores):.4f} (±{np.std(precision_scores):.4f})")
    print(f"Recall: {np.mean(recall_scores):.4f} (±{np.std(recall_scores):.4f})")
    print(f"F1-score: {np.mean(f1_scores):.4f} (±{np.std(f1_scores):.4f})")

    return {
        'precision_mean': np.mean(precision_scores),
        'precision_std': np.std(precision_scores),
        'recall_mean': np.mean(recall_scores),
        'recall_std': np.std(recall_scores),
        'f1_mean': np.mean(f1_scores),
        'f1_std': np.std(f1_scores)
    }

# 4. FUNCIONES DE PRUEBAS DE ESTRÉS
# ------------------------------

def generar_clientes_prueba(n_clientes=50):
    """Genera clientes de prueba con diferentes perfiles"""
    clientes_prueba = []

    segmentos = ['Hogar joven', 'Adulto soltero', 'Familia con niños', 'Adulto mayor', 'Pareja sin hijos']

    for i in range(n_clientes):
        segmento = random.choice(segmentos)

        if segmento == 'Hogar joven':
            edad = random.randint(20, 30)
        elif segmento == 'Adulto soltero':
            edad = random.randint(25, 45)
        elif segmento == 'Familia con niños':
            edad = random.randint(30, 45)
        elif segmento == 'Adulto mayor':
            edad = random.randint(65, 85)
        else:  # Pareja sin hijos
            edad = random.randint(40, 65)

        frecuencia_visitas = round(random.uniform(1, 15), 1)
        gasto_promedio = round(random.uniform(500, 3000), 2)
        usa_cupones = random.randint(0, 1)

        clientes_prueba.append({
            'id_cliente': f'TEST{i+1:03d}',
            'edad': edad,
            'segmento': segmento,
            'frecuencia_visitas_mes': frecuencia_visitas,
            'gasto_promedio_mensual': gasto_promedio,
            'usa_cupones': usa_cupones
        })

    return pd.DataFrame(clientes_prueba)

def generar_compras_prueba(df_clientes_prueba, categorias, todos_productos, num_compras_por_cliente=3):
    """Genera compras de prueba para evaluar el modelo"""
    compras_prueba = []

    # Función para generar canasta por segmento
    def generar_canasta_prueba(segmento, hora):
        n_productos = random.randint(3, 10)

        # Probabilidades adaptadas por segmento y hora como en el script original
        prob_categorias = {
            'Hogar joven': {'Lácteos': 0.6, 'Panadería': 0.7, 'Carnes': 0.4, 'Frutas y Verduras': 0.5, 'Bebidas': 0.8, 'Limpieza': 0.4, 'Despensa': 0.5},
            'Adulto soltero': {'Lácteos': 0.5, 'Panadería': 0.6, 'Carnes': 0.3, 'Frutas y Verduras': 0.4, 'Bebidas': 0.9, 'Limpieza': 0.3, 'Despensa': 0.4},
            'Familia con niños': {'Lácteos': 0.9, 'Panadería': 0.8, 'Carnes': 0.7, 'Frutas y Verduras': 0.7, 'Bebidas': 0.6, 'Limpieza': 0.8, 'Despensa': 0.9},
            'Adulto mayor': {'Lácteos': 0.7, 'Panadería': 0.6, 'Carnes': 0.5, 'Frutas y Verduras': 0.8, 'Bebidas': 0.3, 'Limpieza': 0.6, 'Despensa': 0.7},
            'Pareja sin hijos': {'Lácteos': 0.6, 'Panadería': 0.7, 'Carnes': 0.6, 'Frutas y Verduras': 0.7, 'Bebidas': 0.8, 'Limpieza': 0.5, 'Despensa': 0.6}
        }

        # Modificar por hora
        if hora == 'Mañana':
            prob_categorias[segmento]['Panadería'] += 0.2
        elif hora == 'Tarde':
            prob_categorias[segmento]['Frutas y Verduras'] += 0.1
        else:  # Noche
            prob_categorias[segmento]['Bebidas'] += 0.1

        # Seleccionar categorías
        categorias_seleccionadas = []
        for cat, prob in prob_categorias[segmento].items():
            if random.random() < prob:
                categorias_seleccionadas.append(cat)

        if not categorias_seleccionadas:
            categorias_seleccionadas = [random.choice(list(categorias.keys()))]

        # Seleccionar productos
        canasta = []
        while len(canasta) < n_productos:
            cat = random.choice(categorias_seleccionadas)
            prod = random.choice(categorias[cat])
            if prod not in canasta:
                canasta.append(prod)

        return canasta

    # Fechas de prueba
    fecha_inicio_prueba = datetime(2024, 9, 1)
    fecha_fin_prueba = datetime(2024, 10, 31)
    horas_categoria = ['Mañana', 'Tarde', 'Noche']
    dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

    # Generar compras para cada cliente
    for _, cliente in df_clientes_prueba.iterrows():
        # Generar compras secuenciales con fechas incrementales
        fecha_actual = random.randint(0, (fecha_fin_prueba - fecha_inicio_prueba).days)
        fecha_actual = fecha_inicio_prueba + timedelta(days=fecha_actual)

        for j in range(num_compras_por_cliente):
            # Incrementar fecha entre compras
            dias_incremento = random.randint(1, 14)
            fecha_actual += timedelta(days=dias_incremento)

            if fecha_actual > fecha_fin_prueba:
                break

            hora = random.choice(horas_categoria)
            dia_semana = dias_semana[fecha_actual.weekday()]
            canasta = generar_canasta_prueba(cliente['segmento'], hora)

            # Calcular monto
            base_monto = cliente['gasto_promedio_mensual'] / cliente['frecuencia_visitas_mes']
            monto = round(base_monto * random.uniform(0.7, 1.3), 2)

            compras_prueba.append({
                'id_transaccion': f"TEST{len(compras_prueba)+1:04d}",
                'id_cliente': cliente['id_cliente'],
                'fecha_compra': fecha_actual.strftime('%Y-%m-%d'),
                'hora_compra': hora,
                'dia_semana': dia_semana,
                'productos_comprados': ','.join(canasta),
                'monto_compra': monto,
                'dias_desde_ultima_compra': dias_incremento
            })

    df_compras_prueba = pd.DataFrame(compras_prueba)
    df_compras_prueba['fecha_compra'] = pd.to_datetime(df_compras_prueba['fecha_compra'])
    df_compras_prueba = df_compras_prueba.sort_values(by=['id_cliente', 'fecha_compra'])

    return df_compras_prueba

def preparar_datos_prediccion(cliente, ultima_compra, df_X_entrenamiento, todos_productos):
    """Prepara los datos para predecir la próxima canasta de un cliente"""
    # Crear vector de características
    features_demo = {
        'id_cliente': cliente['id_cliente'],
        'edad': cliente['edad'],
        'segmento': cliente['segmento'],
        'frecuencia_visitas_mes': cliente['frecuencia_visitas_mes'],
        'gasto_promedio_mensual': cliente['gasto_promedio_mensual'],
        'usa_cupones': cliente['usa_cupones'],
        'hora_compra': ultima_compra['hora_compra'],
        'dia_semana': ultima_compra['dia_semana'],
        'monto_compra': ultima_compra['monto_compra'],
        'dias_desde_ultima_compra': ultima_compra['dias_desde_ultima_compra'],
        'productos_comprados': ultima_compra['productos_comprados']
    }

    # Convertir a DataFrame
    df_demo = pd.DataFrame([features_demo])

    # Aplicar one-hot encoding
    productos_one_hot_demo = df_demo['productos_comprados'].apply(lambda x: crear_one_hot_productos(x, todos_productos))
    df_productos_one_hot_demo = pd.DataFrame(productos_one_hot_demo.tolist())

    # One-hot encoding para variables categóricas
    df_demo_encoded = pd.get_dummies(df_demo.drop('productos_comprados', axis=1),
                                    columns=['id_cliente', 'segmento', 'hora_compra', 'dia_semana'])

    # Alinear columnas con el conjunto de entrenamiento
    for col in df_X_entrenamiento.columns:
        if col not in df_demo_encoded.columns and col not in df_productos_one_hot_demo.columns:
            df_demo_encoded[col] = 0

    # Unir one-hot de productos
    missing_producto_cols = set(df_X_entrenamiento.columns) - set(df_demo_encoded.columns) - set(df_productos_one_hot_demo.columns)
    for col in missing_producto_cols:
        df_productos_one_hot_demo[col] = 0

    df_demo_final = pd.concat([df_demo_encoded, df_productos_one_hot_demo], axis=1)

    # Asegurar que tenga exactamente las mismas columnas y orden
    missing_cols = set(df_X_entrenamiento.columns) - set(df_demo_final.columns)
    for col in missing_cols:
        df_demo_final[col] = 0

    df_demo_final = df_demo_final[df_X_entrenamiento.columns]

    return df_demo_final

def ejecutar_prueba_estres(modelo, df_clientes_prueba, df_compras_prueba, df_X_entrenamiento, todos_productos, scaler):
    """Ejecuta una prueba de estrés del modelo con datos de prueba"""
    print("\n==== PRUEBA DE ESTRÉS DEL MODELO ====")

    # Agrupar compras por cliente
    clientes_grupos = df_compras_prueba.groupby('id_cliente')

    predicciones_correctas = 0
    predicciones_totales = 0
    num_productos_predichos = []
    precision_por_cliente = []

    # Para cada cliente
    for id_cliente, grupo in clientes_grupos:
        if len(grupo) < 2:  # Necesitamos al menos 2 compras
            continue

        # Obtener datos del cliente
        cliente_info = df_clientes_prueba[df_clientes_prueba['id_cliente'] == id_cliente].iloc[0]

        # Para cada compra (excepto la última), predecir la siguiente
        for i in range(len(grupo) - 1):
            compra_actual = grupo.iloc[i]
            compra_siguiente = grupo.iloc[i + 1]

            # Preparar datos para predicción
            X_pred = preparar_datos_prediccion(cliente_info, compra_actual, df_X_entrenamiento, todos_productos)

            # Escalar características numéricas
            numeric_cols = ['edad', 'frecuencia_visitas_mes', 'gasto_promedio_mensual', 'monto_compra', 'dias_desde_ultima_compra']
            X_pred[numeric_cols] = scaler.transform(X_pred[numeric_cols])

            # Predecir próxima canasta
            y_pred = modelo.predict(X_pred)

            # Productos predichos
            productos_predichos = [todos_productos[j] for j, val in enumerate(y_pred[0]) if val == 1]

            # Productos reales
            productos_reales = compra_siguiente['productos_comprados'].split(',')

            # Evaluar predicciones
            aciertos = len(set(productos_predichos) & set(productos_reales))

            num_productos_predichos.append(len(productos_predichos))

            if len(productos_predichos) > 0:
                precision_cliente = aciertos / len(productos_predichos)
            else:
                precision_cliente = 0

            precision_por_cliente.append(precision_cliente)

            predicciones_correctas += aciertos
            predicciones_totales += len(productos_predichos)

    # Calcular métricas globales de la prueba
    precision_global = predicciones_correctas / predicciones_totales if predicciones_totales > 0 else 0
    precision_media_cliente = np.mean(precision_por_cliente)
    productos_media = np.mean(num_productos_predichos)

    print(f"Precisión global: {precision_global:.4f}")
    print(f"Precisión media por cliente: {precision_media_cliente:.4f}")
    print(f"Número medio de productos predichos: {productos_media:.2f}")

    return {
        'precision_global': precision_global,
        'precision_media_cliente': precision_media_cliente,
        'productos_media': productos_media
    }

def prueba_con_diferentes_segmentos(modelo, df_X_entrenamiento, todos_productos, categorias, scaler):
    """Evalúa el rendimiento del modelo con diferentes segmentos de clientes"""
    print("\n==== PRUEBA CON DIFERENTES SEGMENTOS ====")

    segmentos = ['Hogar joven', 'Adulto soltero', 'Familia con niños', 'Adulto mayor', 'Pareja sin hijos']
    horas = ['Mañana', 'Tarde', 'Noche']

    resultados_segmentos = []

    for segmento in segmentos:
        print(f"\nEvaluando segmento: {segmento}")

        # Generar cliente fict